<a href="https://colab.research.google.com/github/anfals/neXt-Ray/blob/main/neXt_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Installations


In [ ]:
import argparse
import json
import logging
import os
import re
import shutil
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.checkpoint as cp
import torchvision.transforms as transforms
from PIL import Image
from torch import Tensor
from torch.autograd import Variable
from torch.jit.annotations import List
from torch.utils.data import Dataset, DataLoader
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix, roc_curve, auc
from enum import Enum
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# Data Loaders Setup



In [ ]:
# Data Loaders
train_transformer = transforms.Compose([
    transforms.Resize(320),  # resize the image to 320x320
    transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(), # transform it into a torch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # normalize to weights from ImageNet

# loader for evaluation, no horizontal flip
eval_transformer = transforms.Compose([
    transforms.Resize(320),  # resize the image to 320x320 
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(), # transform it into a torch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # normalize to weights from ImageNet

class Region(Enum):
    WRIST = 1
    SHOULDER = 2
    HUMERUS = 3
    HAND = 4
    FOREARM = 5
    FINGER = 6
    ELBOW = 7

class MuraDataset(Dataset):
    """
    A standard PyTorch definition of Dataset which defines the functions __len__ and __getitem__.
    """

    def __init__(self, data_dir, transform, region=None):
        """
        Store the filenames of the jpgs to use. Specifies transforms to apply on images.

        Args:
            data_dir: (string) directory containing the dataset
            transform: (torchvision.transforms) transformation to apply on image
            region: Region to filter the data files by
        """
        self.filenames = os.listdir(data_dir)
        if region is not None:
            self.filenames = [os.path.join(data_dir, f) for f in self.filenames if f.endswith('.png') and region.name in f]
        else:
            self.filenames = [os.path.join(data_dir, f) for f in self.filenames if f.endswith('.png')]

        self.labels = [int(os.path.split(filename)[-1][0]) for filename in self.filenames]
        self.transform = transform

    def __len__(self):
        # return size of dataset
        return len(self.filenames)

    def __getitem__(self, idx):
        """
        Fetch index idx image and labels from dataset. Perform transforms on image.

        Args:
            idx: (int) index in [0, 1, ..., size_of_dataset-1]

        Returns:
            image: (Tensor) transformed image
            label: (int) corresponding label of image
        """
        image = Image.open(self.filenames[idx])  
        image = self.transform(image)
        return image, self.labels[idx]


def fetch_dataloader(types, data_dir, params):
    """
    Fetches the DataLoader object for each type in types from data_dir.

    Args:
        types: (list) has one or more of 'train', 'val', 'test' depending on which data is required
        data_dir: (string) directory containing the dataset
        params: (Params) hyperparameters

    Returns:
        data: (dict) contains the DataLoader object for each type in types
    """
    dataloaders = {}

    for split in ['train', 'val', 'test']:
        if split in types:
            path = os.path.join(data_dir, "{}".format(split))

            # use the train_transformer if training data, else use transformer without random flip
            if split == 'train':
                dl = DataLoader(MuraDataset(path, train_transformer), batch_size=params.batch_size, shuffle=True,
                                num_workers=params.num_workers,
                                pin_memory=params.cuda)
            elif split =='eval':
                dl = DataLoader(MuraDataset(path, eval_transformer), batch_size=params.batch_size, shuffle=False,
                                num_workers=params.num_workers,
                                pin_memory=params.cuda)
            else:
                for r in Region:
                  dl = DataLoader(MuraDataset(path, eval_transformer, r), batch_size=params.batch_size, shuffle=False,
                                num_workers=params.num_workers,
                                pin_memory=params.cuda)
                  dataloaders[r.name] = dl
                dl = DataLoader(MuraDataset(path, eval_transformer), batch_size=params.batch_size, shuffle=False,
                                    num_workers=params.num_workers,
                                    pin_memory=params.cuda)
            dataloaders[split] = dl
    return dataloaders

# Metrics
All metrics take in the raw outputs from the model and the labels


In [ ]:
def accuracy(outputs, labels):
    outputs = np.rint(outputs)
    return accuracy_score(labels, outputs)

def recall(outputs, labels):
    outputs = np.rint(outputs)
    return recall_score(labels, outputs)

def precision(outputs, labels):
    outputs = np.rint(outputs)
    return precision_score(labels, outputs)  

def f1(outputs, labels):
    outputs = np.rint(outputs)
    return f1_score(labels, outputs)  

def specificity(outputs, labels):
    tn, fp, fn, tp = confusion_matrix(labels, np.rint(outputs), labels=[0,1]).ravel()
    return tn / (tn+fp)

def cohen_kappa(outputs, labels):
    outputs = np.rint(outputs)
    return cohen_kappa_score(outputs, labels)

# maintain all metrics required in this dictionary- these are used in the training and evaluation loops
metrics = {
    'accuracy': accuracy,
    'recall': recall,
    'precision': precision,
    'f1_score': f1,
    'specificity': specificity,
    'cohen': cohen_kappa
}

# Utility Methods


In [ ]:
# Utils
class Params():
    """Class that loads hyperparameters from a json file.

    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        with open(json_path) as f:
          params = json.load(f)
          self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)

    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']"""
        return self.__dict__


class RunningAverage():
    """A simple class that maintains the running average of a quantity

    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)


def set_logger(log_path):
    """Set the logger to log info in terminal and file `log_path`.

    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.

    Example:
    ```
    logging.info("Starting training...")
    ```

    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file

    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)


def save_checkpoint(state, is_best, checkpoint):
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last.pth.tar')
    if not os.path.exists(checkpoint):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth.tar'))


def load_checkpoint(checkpoint, model, optimizer=None):
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    if not os.path.exists(checkpoint):
        raise ("File doesn't exist {}".format(checkpoint))
    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint


# Evaluate Method

In [ ]:
def evaluate(model, loss_fn, dataloader, metrics, params):
    """Evaluate the model

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for data_batch, labels_batch in dataloader:

        # reshape labels batch for our loss function
        labels_batch = labels_batch.type(torch.FloatTensor).reshape((labels_batch.shape[0], 1))

        # move to GPU if available
        if params.cuda:
            data_batch, labels_batch = data_batch.cuda(
                non_blocking=True), labels_batch.cuda(non_blocking=True)
        # fetch the next evaluation batch
        data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)

        # compute model output
        output_batch = model(data_batch)
        #labels_batch = labels_batch.type(torch.FloatTensor).reshape((output_batch.shape[0], 1)).cuda(non_blocking=True)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    logging.info("- Eval metrics : " + metrics_string)
    return metrics_mean

# Training Method


In [ ]:
def train(model, optimizer, loss_fn, dataloader, metrics, params):
    """Train the model 

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()

    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # reshape labels_batch
            labels_batch = labels_batch.type(torch.FloatTensor).reshape((labels_batch.shape[0], 1))

            # move to GPU if available
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(
                    non_blocking=True), labels_batch.cuda(non_blocking=True)
            # convert to torch Variables
            train_batch, labels_batch = Variable(
                train_batch), Variable(labels_batch)

            # compute model output and loss
            output_batch = model(train_batch)
            #labels_batch = labels_batch.type(torch.FloatTensor).reshape((output_batch.shape[0], 1)).cuda(non_blocking=True)
            loss = loss_fn(output_batch, labels_batch)

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            # Evaluate summaries only once in a while
            if i % params.save_summary_steps == 0:
                # extract data from torch Variable, move to cpu, convert to numpy arrays
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # compute all metrics on this batch
                summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.item()
                summ.append(summary_batch)

            # update the average loss
            loss_avg.update(loss.item())

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)
    return metrics_mean


def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, loss_fn, metrics, params, model_dir,
                       restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        val_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches validation data
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from 
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        logging.info("Restoring parameters from {}".format(restore_file))
        load_checkpoint(restore_file, model, optimizer)

    best_val_acc = 0.0
    writer = SummaryWriter(model_dir + "/tensorboard")

    for epoch in range(params.num_epochs):
        # Run one epoch
        logging.info("Epoch {}/{}".format(epoch + 1, params.num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        train_metrics = train(model, optimizer, loss_fn, train_dataloader, metrics, params)

        # Evaluate for one epoch on validation set
        val_metrics = evaluate(model, loss_fn, val_dataloader, metrics, params)

        val_acc = val_metrics['accuracy']
        is_best = val_acc >= best_val_acc

        # Save weights
        save_checkpoint({'epoch': epoch + 1,
                         'state_dict': model.state_dict(),
                         'optim_dict': optimizer.state_dict()},
                        is_best=is_best,
                        checkpoint=model_dir)
        
        # Log the data to tensorboard
        for metric_name in train_metrics:
          writer.add_scalar(metric_name.title() + "/train", train_metrics[metric_name], epoch)
          writer.add_scalar(metric_name.title() + "/validation", val_metrics[metric_name], epoch)
          

        # If best_eval, best_save_path
        if is_best:
            logging.info("- Found new best accuracy")
            best_val_acc = val_acc

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(
                model_dir, "metrics_val_best_weights.json")
            save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(
            model_dir, "metrics_val_last_weights.json")
        save_dict_to_json(val_metrics, last_json_path)
    
    # Close off the SummaryWriter
    writer.flush()
    writer.close()


# ResNet-152 Configuration
Here, we setup our model, a ResNet-152, initialized with weights pretrained on ImageNet and provided by PyTorch. See https://pytorch.org/hub/pytorch_vision_resnet/

We also make some modifications to the final layer, changing into into a sequence of a Dropout layer, a FC layer with one output, and a sigmoid nonlinearity. 

Hyperparamter tuning found freezing the first 148-layers yields the best performance, so this has been done here.

In [ ]:
def load_model(dropout_rate, restore_file=None):
  model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True)
  model.fc = nn.Sequential(
          nn.Dropout2d(p=dropout_rate, inplace=True),
          nn.Linear(2048, 1),
          nn.Sigmoid()
        )
  for param in model.parameters():
    param.requires_grad = False 
  for param in model.fc.parameters():
    param.requires_grad = True
  for param in model.layer4.parameters():
    param.requires_grad = True
  if restore_file:
    load_checkpoint(restore_file, model)
  return model

# Setup and Run

## Retrieve Dataset
The raw MURA Dataset is available here: https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip

In a separate notebook, I downloaded the dataset and did some simple preprocessing on it (resizing images and labeling files more nicely). Here, I connect to Google Drive so I can later retrieve the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# If you wish to download the MURA dataset directly, uncomment the snippet below
# !wget https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
# !unzip MURA-v1.1.zip


## Tensorboard Setup

In [ ]:
%load_ext tensorboard

## Driver to Execute Training and Evaluation on Validation Set


In [ ]:
def run_model(model_dir, data_dir, restore_file):
  # Load the parameters from json file
  json_path = os.path.join(model_dir, 'params.json')
  assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
  params = Params(json_path)

  # use GPU if available
  params.cuda = torch.cuda.is_available()

  # Set the random seed for reproducible experiments
  torch.manual_seed(230)
  if params.cuda:
      torch.cuda.manual_seed(230)

  # Set the logger
  set_logger(os.path.join(model_dir, 'train.log'))

  # Create the input data pipeline
  logging.info("Loading the datasets...")

  # fetch dataloaders
  dataloaders = fetch_dataloader(
      ['train', 'val'], data_dir, params)
  train_dl = dataloaders['train']
  val_dl = dataloaders['val']

  logging.info("- done.")

  # Define the model and optimizer
  model = load_model(params.dropout_rate)
  model = model.to('cuda') if params.cuda else model
  optimizer = optim.Adam(model.parameters(), lr=params.learning_rate, weight_decay=params.weight_decay)

  # fetch loss function and metrics
  loss_fn = torch.nn.BCELoss()

  # Train the model
  logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
  train_and_evaluate(model, train_dl, val_dl, optimizer, loss_fn, metrics, params, model_dir, restore_file)

## Execute Training



In [ ]:
run_model('/content/drive/My Drive/CS 230/Project/experiments/final', 
          '/content/drive/My Drive/CS 230/Project/mura',
          None)

%tensorboard --logdir='/content/drive/My Drive/CS 230/Project/experiments/final'

# Test Set Evaluation
Here is the code we used for evaluating the performance of our final model on the test set, included here for completeness. 

We calculated metrics based on its performance on the test set and drilled down further into the different regions of the body represented in the MURA dataset.



In [ ]:
def get_all_preds_and_labels(model, dataloader, cuda):
  model.eval()
  all_preds, all_labels = torch.tensor([]), torch.tensor([])
  if cuda:
    all_preds, all_labels = torch.tensor([]).cuda(non_blocking=True), torch.tensor([]).cuda(non_blocking=True)
    model = model.to('cuda')
  with tqdm(total=len(dataloader)) as t:
    for batch in dataloader:
        images, labels = batch
        if cuda:
          images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)

        # get predictions
        preds = model(images)

        # concatenate onto the ongoing list
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
        all_labels = torch.cat(
            (all_labels, labels)
            ,dim=0
        )
        t.update()
  return all_preds, all_labels

def calculate_stats(predictions, labels, metrics):
  # compute all metrics on t
  summary_batch = {metric: metrics[metric](predictions, labels)
                         for metric in metrics}
  return summary_batch

def plot_confusion_matrix(predictions, labels, title):
  # calculate confusion matrix 
  matrix = confusion_matrix(labels, np.rint(predictions))
  
  df_cm = pd.DataFrame(matrix, ['negative', 'positive'], ['negative', 'positive'])
  
  plt.figure(figsize=(10,7))
  sn.set(font_scale=1.4) # for label size
  sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g') # font size

  plt.title("Confusion Matrix: " + title)
  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  plt.show()

def plot_auc_roc(predictions, labels):
  fpr, tpr, threshold = roc_curve(labels, predictions)
  roc_auc = auc(fpr, tpr)

  plt.title('Receiver Operating Characteristic')
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()


def get_stats(model, data_dir, params, metrics):
  dataloaders = fetch_dataloader(['test'], data_dir, params)
  predictions, labels = get_all_preds_and_labels(model, dataloaders['test'], params.cuda)

  predictions = predictions.data.cpu().numpy()
  labels = labels.data.cpu().numpy()

  # do the confusion matrix
  plot_confusion_matrix(predictions, labels, "Overall")

  # do the ROC curve
  plot_auc_roc(predictions, labels)

  # print overall metrics
  print("Overall")
  print(calculate_stats(predictions, labels, metrics))

  for region in Region:
    predictions, labels = get_all_preds_and_labels(model, dataloaders[region.name], params.cuda)

    predictions = predictions.data.cpu().numpy()
    labels = labels.data.cpu().numpy()

    print(region.name)
    print(calculate_stats(predictions, labels, metrics))


params = Params('/content/drive/My Drive/CS 230/Project/experiments/final/params.json')
params.cuda = torch.cuda.is_available()
get_stats(load_model(0.5, "/content/drive/My Drive/CS 230/Project/experiments/final/best.pth.tar"), 
               '/content/drive/My Drive/CS 230/Project/mura', 
               params,
               metrics)